In [10]:
import numpy as np
import os

import pickle
import random
import subprocess
from tqdm.notebook import tqdm_notebook

In [11]:
output_folder = '4_10_actualq1'
data_folders = [output_folder]

In [12]:
for data_folder in data_folders:
    try:
        subprocess.check_output(['rm', '-r', '../data/' + data_folder])
    except Exception as e:
        print(e)
    subprocess.check_output(['gsutil', '-m', 'cp', '-r', 'gs://landlord_ai/' + data_folder, '../data/'])

Command '['rm', '-r', '../data/4_10_actualq1']' returned non-zero exit status 1.


In [13]:
directories = ['../data/' + data_folder for data_folder in data_folders]

filenames = []
for directory in directories:
    filenames.extend([directory + '/' + file for file in os.listdir(directory) if file.endswith('.pkl')])

In [14]:
all_history_matrices = []
all_move_vectors = []
all_hand_vectors = []
all_y = []

for filename in tqdm_notebook(filenames):
    with open(filename, 'rb') as f:
        history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
        
    all_history_matrices.extend(history_matrices)
    all_move_vectors.extend(move_vectors)
    all_hand_vectors.extend(hand_vectors)
    all_y.extend(y)

In [15]:
# shuffle all
feature_limit = 75

def densify(sparse_matrix):
    return [x[:feature_limit].todense().astype(np.int8) for x in tqdm_notebook(sparse_matrix)]

all_history_matrices = np.array(densify(all_history_matrices), dtype=np.int8)

In [16]:
all_move_vectors = np.array(all_move_vectors, dtype=np.int8)
all_hand_vectors = np.array(all_hand_vectors, dtype=np.int8)
all_y = np.array(all_y, dtype=np.float32)


In [17]:
np.savez('../data/' + output_folder + '/all.npz', 
         history_matrix=all_history_matrices,
         move_vectors=all_move_vectors,
         hand_vectors=all_hand_vectors,
         y=all_y)